In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

data = loadmat('C:/Users/sarma/Desktop/data')  
data

ValueError: Unknown mat file type, version 32, 50

In [5]:
X = data['X']  
y = data['y']  
X.shape,y.shape


((5000, 400), (5000, 1))

In [14]:
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(sparse=False)
y_onehot=encoder.fit_transform(y)
y_onehot

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [16]:
y[0],y_onehot[0,:]


(array([10], dtype=uint8),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]))

In [79]:
def sigmoid(z):
    return (1/(1+np.exp(-z)))

In [80]:
def forward_propagate(X, theta1, theta2):  
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

In [85]:
def cost(params,insize,hlsize,num_labels,X,y,lr):
    m=X.shape[0]
    X=np.matrix(X)
    y=np.matrix(y)
    theta1=np.matrix(np.reshape(params[:hlsize*(insize+1)],(hlsize,(insize+1))))
    theta2=np.matrix(np.reshape(params[hlsize*(insize+1):],(num_labels,(hlsize+1))))
    
    a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
    
    J=0
    for i in range(m):
        first_term=np.multiply(-y[i,:],np.log(h[i,:]))
        #print(first_term)
        #print(1-h[i,:])
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        #print(second_term)
        J+=np.sum(first_term-second_term)


    J=J/m
    J+=(float(lr)/(2*m))*(np.sum(np.power(theta1[:,1:],2))+np.sum(np.power(theta2[:,1:],2)))
    return J

In [82]:
isize=76800
hsize=260
num_labels=3
lr=1

params=(np.random.random(size=hsize*(isize+1)+num_labels*(hsize+1))-0.5)*0.25
m=X.shape[0]
X=np.matrix(X)
y=np.matrix(y)
theta1=np.matrix(np.reshape(params[:hsize*(isize+1)],(hsize,(isize+1))))
theta2=np.matrix(np.reshape(params[hsize*(isize+1):],(num_labels,(hsize+1))))
theta1.shape,theta2.shape


((64, 401), (10, 65))

In [83]:
a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
a1.shape,z2.shape,a2.shape,z3.shape,h.shape

((5000, 401), (5000, 64), (5000, 65), (5000, 10), (5000, 10))

In [88]:
cost(params,isize,hsize,num_labels,X,y_onehot,lr)

7.6367689679207569

In [89]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z),(1-sigmoid(z)))

In [93]:
def backprop(params,isize,hsize,num_labels,X,y,lr):
    m=X.shape[0]
    X=np.matrix(X)
    y=np.matrix(y)
    theta1=np.matrix(np.reshape(params[:hsize*(isize+1)],(hsize,(isize+1))))
    theta2=np.matrix(np.reshape(params[hsize*(isize+1):],(num_labels,(hsize+1))))
    theta1.shape,theta2.shape
    a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
    J=0
    delta1=np.zeros(theta1.shape)
    delta2=np.zeros(theta2.shape)
    
    for i in range(m):
        first_term=np.multiply(-y[i,:],np.log(h[i,:]))
        #print(first_term)
        #print(1-h[i,:])
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        #print(second_term)
        J+=np.sum(first_term-second_term)


    J=J/m
    J+=(float(lr)/(2*m))*(np.sum(np.power(theta1[:,1:],2))+np.sum(np.power(theta2[:,1:],2)))
    
    
    for t in range(m):
        a1t=a1[t,:]
        z2t=z2[t,:]
        a2t=a2[t,:]
        ht=h[t,:]
        yt=y[t,:]
        
        d3t=ht-yt
        z2t=np.insert(z2t,0,values=np.ones(1))
        d2t=np.multiply((theta2.T*d3t.T).T,sigmoid_gradient(z2t))
        delta1=delta1+(d2t[:,1:]).T*a1t
        delta2=delta2+d3t.T*a2t
        
    delta1=delta1/m
    delta2=delta2/m
    
    delta1[:,1:]=delta1[:,1:]+(theta1[:,1:]*lr)/m
    delta2[:,1:]=delta2[:,1:]+(theta2[:,1:]*lr)/m
    
    grad=np.concatenate((np.ravel(delta1),np.ravel(delta2)))
    return J,grad

In [94]:
J,grad=backprop(params,isize,hsize,num_labels,X,y_onehot,lr)
J,grad.shape

(7.6504888864985379, (26314,))

In [103]:
from scipy.optimize import minimize
fmin=minimize(fun=backprop,x0=params,args=(isize,hsize,num_labels,X,y_onehot,lr),method="TNC",jac=True,options={"maxiter":250})

In [104]:
fmin


     fun: 0.27423818384104715
     jac: array([ -5.18926093e-05,  -6.00362366e-07,   6.19510956e-07, ...,
        -3.12021182e-05,   2.87796452e-05,   1.96511534e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 21
  status: 3
 success: False
       x: array([ 1.08570823, -0.00300181,  0.00309755, ...,  1.84141068,
        0.63722146,  0.41626697])

In [107]:
X=np.matrix(X)
theta1=np.matrix(np.reshape(fmin.x[:hsize*(isize+1)],(hsize,(isize+1))))
theta2=np.matrix(np.reshape(fmin.x[hsize*(isize+1):],(num_labels,(hsize+1))))
a1,z2,a2,z3,h=forward_propagate(X,theta1,theta2)
y_pred=np.array(np.argmax(h,axis=1)+1)
y_pred


array([[10],
       [10],
       [10],
       ..., 
       [ 9],
       [ 9],
       [ 9]], dtype=int64)

In [106]:
correct=[1 if a==b else 0 for (a,b )in zip(y_pred,y)]
accuracy=(sum(map(int,correct))/float(len(correct)))
print (format(accuracy*100))


99.88
